# Eat Safe, Love

## Notebook Set Up

In [ ]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

import json
import numpy as np

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
#query ={"scores.Hygiene": {'$gte': 20}}
query = {'$and': [{"scores.Hygiene": {'$gte': 20}}, {"scores.Hygiene": {'$lte': 20}}]}

# Use count_documents to display the number of documents in the result
count=establishments.count_documents(query)
print(count)
# Display the first document in the results using pprint
results = establishments.find_one(query)
pprint(results)

In [ ]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query)
Hygiene_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print(Hygiene_df.shape)
# Display the first 10 rows of the DataFrame
Hygiene_df.head(5)

In [ ]:
Hygiene_df.info()

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query1 = {'LocalAuthorityName': {'$regex':'London'}, 'RatingValue': {'$gte': 4}}
print("Number of documents in result: ", establishments.count_documents(query1))
print("First result:")
results1 = establishments.find(query1)
pprint(results1[0])

In [ ]:
# Convert the result to a Pandas DataFrame
df2 = pd.DataFrame(establishments.find(query1))

# Display the number of rows in the DataFrame
print(f"There are {len(df2)} rows in this DataFrame.")

# Display the first 10 rows of the DataFrame
df2.head(10)



### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Create the pipeline: match, then sort, then limit to 5
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

query3 = {'RatingValue': 5, 
         'geocode.latitude': {'$lte': (latitude + degree_search), '$gte': (latitude - degree_search)}, 
         'geocode.longitude': {'$lte': (longitude + degree_search), '$gte': (longitude - degree_search)} 
         }
sort =  [('scores.Hygiene', 1)]

# Print the results
for result3 in establishments.find(query3).sort(sort).limit(5):
    pprint(result3)

In [ ]:
# Convert result to Pandas DataFrame
lat_long=pd.DataFrame(establishments.find(query3).sort(sort).limit(5))
lat_long

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
match_query = {'$match': {'scores.Hygiene': 0}}
group_query = {'$group': {'_id': '$LocalAuthorityName', 
                          'count': {'$sum': 1}
                         }
              }
sort_query = {'$sort': {'count': -1}}
pipeline = [match_query, group_query, sort_query]
results4 = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"There are {len(results4)} documents in the result.")

# Print the first 10 results
pprint(results4[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
agg_df = pd.json_normalize(results4)

# Display the number of rows in the DataFrame
print(f"There are {len(agg_df)} rows in the DataFrame.")

# Display the first 10 rows of the DataFrame
agg_df.head(10)